In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Data/IC_dataset')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from model_A import ImageCaptioner,EncoderCNN, DecoderRNN
import os
import csv
import zipfile
import pandas as pd

In [3]:
MODEL_DIR = "/content/drive/MyDrive/Colab Notebooks/Data/IC_dataset"
BEST_MODEL = "trained_model_A.pth"
saved_model_path=os.path.join(MODEL_DIR,BEST_MODEL)
vocab_path=r"/content/drive/MyDrive/Colab Notebooks/Data/IC_dataset/vocabulary.json"
# model = VisionGPT2Model(model_config)  # Make sure to provide the necessary configuration
# model.load_state_dict(torch.load(saved_model_path))

In [4]:
zip_file_path=r"/content/drive/MyDrive/Colab Notebooks/Data/IC_dataset/test.zip"
extract_to="/content/drive/MyDrive/Colab Notebooks/Data/IC_dataset"
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents of the zip file to the specified directory
    zip_ref.extractall(extract_to)

In [8]:
test_img_path=r"/content/drive/MyDrive/Colab Notebooks/Data/IC_dataset/test"
files = sorted(os.listdir(test_img_path),key=lambda x: int(x[5:-4]))

# Initialize an empty DataFrame
df = pd.DataFrame(columns=['filename', 'path'])

# Iterate over the files
for i, file in enumerate(files, start=1):
    # Construct the file path
    file_path = os.path.join(test_img_path, file)
    # print(file_path)
    # Append the filename and path to the DataFrame
    new_row={'filename': f'test_{i}', 'path': file_path}
    df.loc[len(df)] = new_row

df.head(5)

,filename,path
0,test_1,/content/drive/MyDrive/Colab Notebooks/Data/IC...
1,test_2,/content/drive/MyDrive/Colab Notebooks/Data/IC...
2,test_3,/content/drive/MyDrive/Colab Notebooks/Data/IC...
3,test_4,/content/drive/MyDrive/Colab Notebooks/Data/IC...
4,test_5,/content/drive/MyDrive/Colab Notebooks/Data/IC...


In [9]:
output_df = pd.DataFrame(columns=['filename', 'caption'])

model=ImageCaptioner(vocab_path,saved_model_path)
for index, row in df.iterrows():
    filename = row['filename']
    path = row['path']
    # print(f"Filename: {filename}, Path: {path}")
    out=model.predict(path)
    new_row={'filename': f'test_{index}', 'caption': out}
    output_df.loc[len(output_df)] = new_row


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
output_df['filename'] = ['test_' + str(i + 1) for i in range(len(output_df))]

In [12]:
output_df.head(5)

,filename,caption
0,test_1,outside outside outside outside outside outsid...
1,test_2,most most most most most most most most most m...
2,test_3,outside outside outside outside outside outsid...
3,test_4,outside outside outside outside outside outsid...
4,test_5,<START> bushes bushes bushes bushes bushes bus...


In [13]:
output_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Data/IC_dataset/output_A.csv', index=False)